# 训练教程


## Install Dependencies

In [ ]:
!pip install transformers==4.28.1
!pip install git+https://github.com/huggingface/peft.git@13e53fc
!pip install datasets
!pip install sentencepiece
!pip install deepspeed

## 克隆代码





In [ ]:
!git clone https://github.com/IMOSR/Media-LLaMA.git

## 指令微调 Alpaca-7B

参考chinese llama的博客 https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/SFT-Script 设置对应的参数

In [ ]:
# 对应目录记得改成自己的

!cd [脚本地址]/train && torchrun --nnodes 1 --nproc_per_node 1 run_clm_sft_with_peft.py \
    --deepspeed ds_zero2_no_offload.json \
    --model_name_or_path decapoda-research/llama-7b-hf \
    --tokenizer_name_or_path ziqingyang/chinese-alpaca-lora-7b \
    --dataset_dir [数据地址]/tiktok_data \
    --validation_split_percentage 0.001 \
    --per_device_train_batch_size 1 \
    --do_train \
    --fp16 \
    --seed $RANDOM \
    --max_steps 10000 \
    --lr_scheduler_type cosine \
    --learning_rate 1e-4 \
    --warmup_ratio 0.03 \
    --weight_decay 0 \
    --logging_strategy steps \
    --logging_steps 10000 \
    --save_strategy steps \
    --save_total_limit 2 \
    --save_steps 10000 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 8 \
    --max_seq_length 512 \
    --output_dir [输出地址] \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --torch_dtype float16 \
    --peft_path [peft 模型的地址] \
    --gradient_checkpointing \
    --ddp_find_unused_parameters False

# 合并模型
参考Chinese llama的文档：https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/%E5%9C%A8%E7%BA%BF%E6%A8%A1%E5%9E%8B%E5%90%88%E5%B9%B6%E4%B8%8E%E8%BD%AC%E6%8D%A2

In [ ]:
!python merge/merge_llama_with_chinese_lora.py \
    --base_model  decapoda-research/llama-7b-hf\
    --lora_model ziqingyang/chinese-llama-plus-lora-7b,[你训练完的模型的地址]/sft_lora_model \
    --output_type pth \
    --output_dir /content/drive/MyDrive/gitcode/sample_data/llama-combined-ch-7

# llama.cpp量化部署


## 克隆代码和编译

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

In [ ]:

!cd /content/drive/MyDrive/gitcode/llama.cpp && make

## 模型转换为ggml格式（FP16）

In [ ]:
!cd /content/drive/MyDrive/gitcode/llama.cpp && python convert.py /content/drive/MyDrive/gitcode/sample_data/llama-combined-ch-7

## 测试模型

In [ ]:
!cd /content/drive/MyDrive/gitcode/llama.cpp && chmod 777 main && ./main -m /content/drive/MyDrive/gitcode/sample_data/llama-combined-ch-7/ggml-model-f16.bin --color -f ./prompts/alpaca.txt -ins -c 2048 --temp 0.2 -n 256 --repeat_penalty 1.1


main: build = 649 (98ed165)
main: seed  = 1688020234
llama.cpp: loading model from /content/drive/MyDrive/gitcode/sample_data/llama-combined-ch-7/ggml-model-f16.bin
llama_model_load_internal: format     = ggjt v1 (pre #1405)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.07 MB
llama_model_load_internal: mem required  = 14925.62 MB (+ 1026.00 MB per state)
.
llama_init_from_file: kv self size  = 1024.00 MB

system_info: n_threads = 6 / 12 | AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_

## 如果想量化模型请参考
https://github.com/ymcui/Chinese-LLaMA-Alpaca/blob/main/notebooks/convert_and_quantize_chinese_llama_and_alpaca.ipynb